In [1]:
# default_exp actorcritic.diayn

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
#export
import torch.nn.utils as nn_utils
from fastai.torch_basics import *
import torch.nn.functional as F
from fastai.data.all import *
from fastai.basics import *
from dataclasses import field,asdict
from typing import List,Any,Dict,Callable
from collections import deque
import gym
import torch.multiprocessing as mp
from torch.optim import *
from dataclasses import dataclass

from fastrl.data import *
from fastrl.async_data import *
from fastrl.basic_agents import *
from fastrl.learner import *
from fastrl.metrics import *
from fastai.callback.progress import *
from fastrl.ptan_extension import *
from fastrl.actorcritic.sac import *

from torch.distributions import *

if IN_NOTEBOOK:
    from IPython import display
    import PIL.Image

/opt/conda/envs/fastrl/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729047590/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:
# hide
from nbdev.showdoc import *
from nbdev.imports import *
from nbdev.export2html import *
if not os.environ.get("IN_TEST", None):
    assert IN_NOTEBOOK
    assert not IN_COLAB
    assert IN_IPYTHON

# DIAYN

> Diversity Is All You Need

## Modules

In [7]:
env='Pendulum-v0'
agent=SAC(3,gym.make(env).action_space,gamma=0.99,tau=0.005,alpha=0.2)

block=FirstLastExperienceBlock(agent=agent,seed=0,n_steps=2,exclude_nones=True,
                               dls_kwargs={'bs':1,'num_workers':0,'verbose':False,'indexed':True,'shuffle_train':False})
blk=IterableDataBlock(blocks=(block),
                      splitter=FuncSplitter(lambda x:False),
#                       batch_tfms=lambda x:(x['s'],x),
                     )
dls=blk.dataloaders([env]*1,n=1000,device=default_device())

learner=SACLearner(dls,agent=agent,cbs=[ExperienceReplay(sz=1000000,bs=64,starting_els=1000,max_steps=gym.make(env)._max_episode_steps),SACCriticTrainer],
                   metrics=[AvgEpisodeRewardMetric(experience_cls=ExperienceFirstLast)])
learner.fit(30,lr=0.001,wd=0)

epoch,train_loss,train_avg_episode_r,valid_loss,valid_avg_episode_r,time
0,68.240349,-1428.730126,None,-1428.730126,00:15
1,140.679016,-1573.647103,None,-1573.647103,00:15
2,230.889587,-1502.739255,None,-1502.739255,00:15
3,333.508118,-1456.170725,None,-1456.170725,00:18
4,343.003479,-1406.864173,None,-1406.864173,00:15
5,375.362946,-1375.714182,None,-1375.714182,00:15
6,407.716522,-1340.233665,None,-1340.233665,00:16
7,443.068268,-1291.501812,None,-1291.501812,00:15
8,473.301788,-1249.785071,None,-1249.785071,00:15
9,494.265442,-1215.713414,None,-1215.713414,00:16


In [ ]:
# hide
from nbdev.export import *
from nbdev.export2html import *
notebook2script()
notebook2html()